# Workshop 06

<b>Obiettivo</b>: Tradurre il codice del modello geometrico di <i>S. Stefano Rotondo al Celio</u>.

In [1]:
# Importo le librerie:
import sys
sys.path.insert(0, 'C:\Users\Camilla Bianca\Documents\GGPL\lar-cc\larlib')
from larlib import *
import math

Evaluating fenvs.py..
...fenvs.py imported in 0.0154095757735 seconds


<hr>

In [ ]:
# PLASM:
%=============================================================%
%                                                             %
%       Modello geometrico di S. Stefano Rotondo al Celio     %
%               Modello della Gerusalemme Celeste             %
%                                                             %
%                                                             %
%                       A.Paoluzzi, 1998                      %
%                                                             %
%=============================================================%

%*************************************************************%
%---Misure di riferimento--(cubito = 0.462 m)-----------------%
%*************************************************************%

%-- 1 cubito = 444 mm ----------------------------------------%
DEF r0 = 24; %---cubiti; raggio interno del muro interno------%
DEF r2 = 48; %---cubiti; raggio esterno del muro medio--------%
DEF r3 = 72; %---cubiti; raggio esterno del muro esterno------%
DEF r4 = 88; %---cubiti; raggio esterno scale esterne---------%

DEF wsteps = 10;  %---cubiti; profondita' scale esterne-------%
DEF nsteps = 10;  %---numero alzate/pedate scale esterne------%
DEF wstep = wsteps/nsteps; %---cubiti; larghezza pedata-------%
DEF hstep = 1/3; %---cubiti; altezza alzata-------------------%
DEF hw4 = 29; %--cubiti; altezza all'imposta del muro esterno-%
DEF hbasament = hstep*nsteps;

In [2]:
# PYPLASM:
r0 = 24.0
r2 = 48.0
r3 = 72.0
r4 = 88.0

wsteps = 10.0
nsteps = 10.0
wstep = wsteps/nsteps
hstep = 1.0/3.0
hw4 = 29.0
hbasament = hstep*nsteps

<hr>

In [ ]:
# PLASM:
%-------------------------------------------------------------%
%---Mappa cilindrica------------------------------------------%
%-------------------------------------------------------------%

DEF cylMap = MAP:[S2 * sin ~ S1, S2 * cos ~ S1, S3];

In [3]:
# PYPLASM:
import math 
fx = lambda x : x[1] * sin(x[0])
fy = lambda x : x[1] * cos(x[0])
fz = lambda x : x[2]

cylMap = MAP([fx,fy,fz])

<hr>

In [ ]:
# PLASM:
%-------------------------------------------------------------%
%---Scala esterna---------------------------------------------%
%-------------------------------------------------------------%

DEF vdom (h::IsRealPos) = ((EMBED:1 ~ Intervals:(PI*3/24)):3) * QUOTE:<h>;
DEF hdom (w::IsRealPos) = (S:2:-1 ~ EMBED:1):(Intervals:(PI*3/24):3 * QUOTE:<w>);

DEF steps (w,h::IsRealPos) (n::IsIntPos) = (STRUCT ~ CAT ~ #:n):
  < vdom:h, T:3:h, hdom:w, T:2:(-:w) >;

DEF hbasament = hstep*nsteps;
DEF stair = (MKPOL ~ UKPOL):(steps:<wstep,hstep>:nsteps  
              RIGHT (S:2:-1 ~ @2 ~ CUBOID):<PI/24,wsteps,hbasament>);
DEF ramp  = cylMap:(T:2:r4:stair);
DEF stairs = (STRUCT ~ ##:12):<ramp, R:<1,2>:(PI/6)>;

In [4]:
# PYPLASM:
def vdom(h):
    return PROD([COMP([EMBED(1),INTERVALS(PI*3/24)])(3),QUOTE([h])])

def hdom(w):
    return COMP([S(2)(-1),EMBED(1)])(PROD([INTERVALS(PI*3/24)(3),QUOTE([w])]))

def steps(w,h,n):
    return COMP([STRUCT,CAT,N(n)])([vdom(h),T(3)(h),hdom(w),T(2)(-w)])

hbasament = hstep*nsteps

stair = RIGHT([COMP([MKPOL,UKPOL])(steps(wstep,hstep,nsteps)),
               COMP([S(2)(-1),SKEL_2,CUBOID])([PI/24,wsteps,hbasament])])
ramp = cylMap(T(2)(r4)(stair))
stairs = STRUCT(NN(12)([ramp,R([1,2])(PI/6)]))

<hr>

In [ ]:
# PLASM:
%-------------------------------------------------------------%
%---Basamento-------------------------------------------------%
%-------------------------------------------------------------%

DEF basisSector = (cylMap ~ EMBED:1):
    (Intervals:(2*PI/12):4 * Intervals:(r4 - (wstep*(nsteps - 1))):1);
DEF basis = (STRUCT ~ ##:12):<basisSector, R:<1,2>:(2*PI/12)>;
DEF basement = (R:<1,2>:(PI/-48) ~ STRUCT):< stairs, T:3:(hstep*nsteps), basis >;

%
basement
%

In [5]:
# PYPLASM:
basisSector = COMP([cylMap,EMBED(1)])(PROD([INTERVALS(2*PI/12)(4),INTERVALS(r4-(wstep*(nsteps-1)))(1)]))
basis = STRUCT(NN(12)([basisSector,R([1,2])(2*PI/12)]))
basement = COMP([R([1,2])(PI/-48),STRUCT])([stairs,T(3)(hstep*nsteps),basis])

VIEW(basement)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E339420> >

<hr>

In [ ]:
# PLASM:
%-------------------------------------------------------------%
%---Muro esterno----------------------------------------------%
%-------------------------------------------------------------%

DEF ExtWall2Da = MKPOL:<<<0,0>,<7,0>,<7,5>,<0,8>,<7,3>,<9,4>,
<10.5,1.5>,<10.5,0>,<11,1.5>,<11,3>>,<1..4,<2,8,7,6,5>,<6,7,9,10>>,
<<1>,<2>,<3>>> & ((Q ~ #:12):1 * Q:8);
DEF ExtWall2Db = MKPOL:<<<0.5,0>,<4,0>,<5,0>,<6,0>,<6,3>,<5.5,3.5>,
  <5,3>,<4.5,3.5>,<4,3>,<2,4>,<0,3>,<0,1.5>,<0.5,1.5>>,
  <<10,11,12,13>,<1,2,9,10,13>,<2,3,7,8,9>,<3,4,5,6,7>>,<1..4>> 
&  ((Q ~ #:6):1 * Q:4);
DEF ExtWall2D = STRUCT:<ExtWall2Da, T:1:11, ExtWall2Db>;
DEF sizxExtWall = SIZE:1:ExtWall2D;

DEF ExtWall = R:<2,3>:(PI/2):(ExtWall2D * QUOTE:<1.5>);
DEF CurvedExtWall = (cylMap ~ T:2:r3 ~ S:<1,3>:<PI/(4*sizxExtWall),hw4/8>):   ExtWall;
DEF DoubleExtWall = STRUCT:<CurvedExtWall, S:1:-1, CurvedExtWall>;
DEF FullExtWall = (STRUCT ~ ##:4):<DoubleExtWall, R:<1,2>:(PI/2)>;

%
STRUCT:<basement, T:3:hbasament, FullExtWall>;
%

In [6]:
# PYPLASM:
ExtWall2Da = INTERSECTION([
    MKPOL([[[0,0],[7,0],[7,5],[0,8],[7,3],[9,4],
            [10.5,1.5],[10.5,0],[11,1.5],[11,3]],
            [range(1,5),[2,8,7,6,5],[6,7,9,10]],[[1],[2],[3]]]),
    PROD([COMP([QUOTE,N(12)])(1),Q(8)])
])
ExtWall2Db = INTERSECTION([
    MKPOL([[[0.5,0],[4,0],[5,0],[6,0],[6,3],[5.5,3.5],
            [5,3],[4.5,3.5],[4,3],[2,4],[0,3],[0,1.5],[0.5,1.5]],
            [[10,11,12,13],[1,2,9,10,13],[2,3,7,8,9],[3,4,5,6,7]],
            range(1,5)]),
    PROD([COMP([QUOTE,N(6)])(1),Q(4)])
])
ExtWall2D = STRUCT([ExtWall2Da,T(1)(11),ExtWall2Db])
sizxExtWall = SIZE(1)(ExtWall2D)
ExtWall = R([2,3])(PI/2)(PROD([ExtWall2D,QUOTE([1,5])]))
CurvedExtWall = COMP([cylMap,T(2)(r3),S([1,3])([PI/(4*sizxExtWall),hw4/8])])(ExtWall)
DoubleExtWall = STRUCT([CurvedExtWall,S(1)(-1),CurvedExtWall])
FullExtWall = COMP([STRUCT,NN(4)])([DoubleExtWall,R([1,2])(PI/2)])
VIEW(FullExtWall)
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E3881E0> >

<hr>

In [ ]:
# PLASM:
%-------------------------------------------------------------%
%---Colonne intermedie----------------------------------------%
%-------------------------------------------------------------%

DEF Column (w,h::IsRealPos) = basis TOP trunk TOP capitel
WHERE
  basis = CUBOID:<w,w,2*w/3>,
  trunk = CYLINDER:<w/2*0.85, h - w>:8,
  capitel = CUBOID:<w,w,w/3>
END;
DEF arcAngle = 2*PI/50.4;
DEF wallAngle = -3.2*arcAngle/4;
DEF RotCross = 3.2*arcAngle/4 + 2.5*arcAngle;

DEF MyColumn = (MKPOL ~ UKPOL ~ T:2:((r2 - 0.75)) ~ Column):<1.5,hCol>;
DEF The4cols = (R:<1,2>:(arcAngle*0.4/3.2) ~ MKPOL ~ UKPOL ~ 
  STRUCT ~ ##:4):< R:<1,2>:(-:arcAngle), MyColumn >;
DEF The5cols = (R:<1,2>:(arcAngle*0.4/3.2) ~ MKPOL ~ UKPOL ~ 
  STRUCT ~ ##:5):< R:<1,2>:(-:arcAngle), MyColumn >;
DEF TheBotWal = (cylMap ~ MKPOL ~ UKPOL ~ T:2:(r2 - 0.75) ~ CUBOID):
  < 3.2*arcAngle/4, 1.5, hCol >;
DEF TheSecCols = STRUCT:< R:<1,2>:RotCross,
  TheBotWal, R:<1,2>:wallAngle, 
  The4cols, R:<1,2>:(-5*arcAngle), 
  TheBotWal, R:<1,2>:wallAngle, 
  The5cols >;
DEF TheMedColumns = (STRUCT ~ ##:4):<TheSecCols, R:<1,2>:(PI/-2)>;
DEF hCol = 12 ;

%
STRUCT:<basement, T:3:hbasament, FullExtWall, TheMedColumns >;
%

In [7]:
# PYPLASM:
def Column(argomento):
    w,h = argomento
    basis = CUBOID([w,w,2*w/3])
    trunk = CYLINDER([w/2*0.85,h-w])(8)
    capitel = CUBOID([w,w,w/3])
    return TOP([TOP([trunk,basis]),capitel])

hCol = 12
arcAngle = 2*PI/50.4
wallAngle = -3.2*arcAngle/4
RotCross = 3.2*arcAngle/4 + 2.5*arcAngle;
MyColumn = COMP([MKPOL,UKPOL,T(2)(r2-0.75),Column])([1.5,hCol])
The4cols = COMP([R([1,2])(arcAngle*0.4/3.2),MKPOL,UKPOL,STRUCT,NN(4)])([R([1,2])(-arcAngle),MyColumn])
The5Cols = COMP([R([1,2])(arcAngle*0.4/3.2),MKPOL,UKPOL,STRUCT,NN(5)])([R([1,2])(-arcAngle),MyColumn])
TheBotWal = COMP([cylMap,MKPOL,UKPOL,T(2)(r2-0.75),CUBOID])([3.2*arcAngle/4,1.5,hCol])
TheSecCols = STRUCT([R([1,2])(RotCross),
                    TheBotWal,R([1,2])(wallAngle),
                    The4cols,R([1,2])(-5*arcAngle),
                    TheBotWal,R([1,2])(wallAngle),
                    The5Cols])
TheMedColumns = COMP([STRUCT,NN(4)])([TheSecCols,R([1,2])(PI/(-2))])
VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,TheMedColumns]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E3D82A0> >

<hr>

In [ ]:
# PLASM:
%-------------------------------------------------------------%
%---Archi sulle colonne---------------------------------------%
%-------------------------------------------------------------%

DEF bottomArc (d::IsRealPos) = Bezier:S1:<<0,0>,<0,2*d/3>,<d,2*d/3>,<d,0>>;
DEF topArc (d::IsRealPos) = Bezier:S1:<<0,2*d/3>,<d,2*d/3>>;
DEF arc2D  (d::IsRealPos) = Bezier:S2:<bottomArc:d,topArc:d>;
DEF arc3D  (d::IsRealPos)(w::IsRealPos) = (T:2:w ~ R:<2,3>:(PI/2)):
  ( MAP:(CONS:(arc2D:d)):(Intervals:1:8 * Intervals:1:1) * QUOTE:<w> );
DEF Interarc (d1,d2::IsReal)(w::IsRealPos)= CUBOID:<d1,w,2*d2/3>;
DEF Xarc (d1,d2::IsRealPos) (w::IsRealPos) = 
  Interarc:<d1,d2>:w RIGHT arc3D:d2:w RIGHT Interarc:<d1,d2>:w;

DEF SizeArc = SIZE:1:TheArc;
DEF TheArc = Xarc:<0.4,3.2>:1.5;
DEF HeigthArc = SIZE:3:CurvedArc;
DEF CurvedArc = (cylMap ~ MKPOL ~ UKPOL ~ T:2:(r2 - 0.75) ~ S:<1,3>:<sx,sz>):TheArc
WHERE
  sx = (1/SizeArc)*arcAngle,
  sz = sx * (r2 - 2)
END;
DEF The5arcs = (MKPOL ~ UKPOL ~ STRUCT ~ ##:5):< CurvedArc, R:<1,2>:(-:arcAngle) >;
DEF The6arcs = (MKPOL ~ UKPOL ~ STRUCT ~ ##:6):< CurvedArc, R:<1,2>:(-:arcAngle) >;
DEF TheMidWall = (cylMap ~ MKPOL ~ UKPOL ~ T:2:(r2 - 0.75) ~ CUBOID):
  <3.2*arcAngle/4, 1.5, HeigthArc>;
DEF TheSector = STRUCT:< R:<1,2>:RotCross,
  TheMidWall, R:<1,2>:wallAngle, 
  The5arcs, R:<1,2>:(-5*arcAngle), 
  TheMidWall, R:<1,2>:wallAngle, 
  The6arcs >;
DEF TheArcs = (STRUCT ~ ##:4):<TheSector, R:<1,2>:(PI/-2)>;
DEF TheArcsCols = STRUCT:< TheMedColumns, T:3:hCol, TheArcs >;

%
STRUCT:<basement, T:3:hbasament, FullExtWall, TheArcsCols >;
%

In [10]:
# PYPLASM:
def bottomArc(d):
    return BEZIER(S1)([[0,0],[0,2*d/3],[d,2*d/3],[d,0]])

def topArc(d):
    return BEZIER(S1)([[0,2*d/3],[d,2*d/3]])

def arc2D(d):
    return BEZIER(S2)([bottomArc(d),topArc(d)])

def arc3D(d,w):
    return COMP([T(2)(w),R([2,3])(PI/2)])(MAP(arc2D(d))(PROD([PROD([INTERVALS(1)(8),INTERVALS(1)(1)]),QUOTE([w])])))

def Interarc(d1,d2,w):
    return CUBOID([d1,w,2*d2/3])

def Xarc(d1,d2,w):
    return RIGHT([RIGHT([Interarc(d1,d2,w),arc3D(d2,w)]),Interarc(d1,d2,w)])

def CurvedArc():
    sx = (1/SizeArc)*arcAngle
    sz = sx*(r2-2)
    return COMP([cylMap,MKPOL,UKPOL,T(2)(r2-0.75),S([1,3])([sx,sz])])(TheArc)

TheArc = Xarc(0.4,3.2,1.5)
SizeArc = SIZE(1)(TheArc)
HeightArc = SIZE(3)(CurvedArc())
The5arcs = COMP([MKPOL,UKPOL,STRUCT,NN(5)])([CurvedArc(),R([1,2])(-arcAngle)])
The6arcs = COMP([MKPOL,UKPOL,STRUCT,NN(6)])([CurvedArc(),R([1,2])(-arcAngle)])
TheMidWall = COMP([cylMap,MKPOL,UKPOL,T(2)(r2-0.75),CUBOID])([3.2*arcAngle/4,1.5,HeightArc])
TheSector = STRUCT([
    R([1,2])(RotCross),
    TheMidWall,R([1,2])(wallAngle),
    The5arcs,R([1,2])(-5*arcAngle),
    TheMidWall,R([1,2])(wallAngle),
    The6arcs
])
TheArcs = COMP([STRUCT,NN(4)])([TheSector,R([1,2])(PI/(-2))])
TheArcsCols = STRUCT([TheMedColumns,T(3)(hCol),TheArcs])

VIEW(STRUCT([basement,T(3)(hbasament),FullExtWall,TheArcsCols]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E388EA0> >

<hr>